# Produces dataset of project times from appended pipeline datasets ("all_quarters_merged.csv")

In [20]:
import pandas as pd
import logging
import dateutil
from dateutil import parser
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', 500)
pd.set_option('display.float_format', lambda x: '%.2f' % x) 

In [2]:
logging.basicConfig(level=logging.INFO)

# Code Broken Down

In [33]:
df = pd.read_csv("cleaned/all_quarters_merged.csv")

/Users/briangoggin/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (1,2,6,8,12,13,16,17,20,21,22,23,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,46,48,49,52,53,54,56,61,63,64,65,66,67,68,69,70,72,76,79,80,81,82,83,85,86,87,88,89,90,91,92,93,94,95,100,102,104,105,106,107,108,110,111,112,113,114,117,119,126,129,131,132,133,134,135,136) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [34]:
# keep only variables of interest
list = ['address', 'apn', 'best_date', 'best_stat','firstfiled', 'report_quarter', 'report_year', 'units', 'unitsnet', 'dbi_permit', 'x', 'y']
df = df[list]

In [35]:
#consolidate status categories. Start with 3 for now
def status_function(value):
    if value['best_stat']=="CONSTRUCTION":
        field = 'Under Construction'
    elif (value['best_stat']=='BP APPROVED') | (value['best_stat']=='BP ISSUED') | (value['best_stat']=='BP REINSTATED'):
        field = 'Building Permit Approved'
    else:
        field = 'Proposed'
    return field
        
df['status']=df.apply(status_function, axis=1)

In [36]:
gb = df.groupby(['apn', 'address'])

In [37]:
gb.groups

{('3726103',
  '119 07TH ST'): Int64Index([21463, 22631, 23930, 26541, 26574, 27974], dtype='int64'),
 ('3648033', '3669-3671 23RD ST'): Int64Index([20791], dtype='int64'),
 ('3730015',
  '1140 FOLSOM ST'): Int64Index([21324, 22610, 23910, 26524, 26748, 28225], dtype='int64'),
 ('1183010',
  '601 BRODERICK ST'): Int64Index([10126, 10909, 11641, 12434, 13102, 14051, 14871, 15739, 16536,
             17836, 18857, 19136, 20724, 21845, 23557],
            dtype='int64'),
 ('0103032',
  '140 JASPER PL'): Int64Index([ 2862,  4097,  4669,  5709,  6346,  7111,  7981,  8707,  8976,
             10217, 10734, 11464, 12438, 12936, 13847, 14701, 15575, 16412],
            dtype='int64'),
 ('6959030',
  '1927 ALEMANY BL'): Int64Index([ 4924,  5387,  6384,  7221,  7793,  8162,  8902, 10231, 10798,
             11519, 12285, 12990, 13899, 14750, 15615, 16456, 17826, 18831,
             19529, 20680, 22026, 22779, 23880, 25514, 26928, 28390],
            dtype='int64'),
 ('6652026',
  '489 30TH ST'):

In [87]:
group_df = gb.get_group(('3705039','55 05TH ST'))

In [88]:
group_df = group_df.sort('best_date')
group_df.reset_index(drop=True, inplace=True)

/Users/briangoggin/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [89]:
group_df

,address,apn,best_date,best_stat,firstfiled,report_quarter,report_year,units,unitsnet,dbi_permit,x,y,status
0,55 05TH ST,3705039,2012-06-20,BP FILED,2012-06-20 00:00:00,2,2012,116.00,8.00,201206202986,37.78,-122.41,Proposed
1,55 05TH ST,3705039,2012-10-26,BP APPROVED,2012-06-20,3,2012,116.00,8.00,201206202986,-122.41,37.78,Building Permit Approved
2,55 05TH ST,3705039,2012-12-12,CONSTRUCTION,2012-06-20,4,2012,116.00,8.00,201206202986,37.78,-122.41,Under Construction


In [ ]:
# identify building permit ID. Then fill in rest of quarters with this permit ID.
building_permit=np.nan
for index, row in group_df.iterrows():
    if pd.isnull(row['dbi_permit']) and not pd.isnull(building_permit):
        building_permit=building_permit
    else:
        building_permit = row['dbi_permit']

In [ ]:
building_permit

In [ ]:
group_df['dbi_permit']=building_permit

In [ ]:
#Identify completion quarter
for index, row in group_df.iterrows():
    if row['best_stat'] !='CONSTRUCTION':
        comp_quarter= np.nan
        comp_year=np.nan
    elif row['best_stat'] == 'CONSTRUCTION':
        comp_quarter = row['report_quarter']
        comp_year = row['report_year']

if pd.isnull(comp_quarter):
    pass
if comp_quarter == 4 and comp_year == 2016:
    comp_quarter = np.nan
elif comp_quarter == 4:
    comp_quarter = 1
    comp_year = 1+comp_year
else:
    comp_quarter = 1+comp_quarter
    

In [ ]:
if comp_quarter ==1:
    comp_daymth= '01/01'
elif comp_quarter ==2:
    comp_daymth= '04/01'
elif comp_quarter==3:
    comp_daymth= '07/01'
elif comp_quarter == 4:
    comp_daymth= '10/01'
elif pd.isnull(comp_quarter):
    comp_daymth=np.nan
    
if pd.isnull(comp_daymth):
    comp_date = np.nan
else:
    comp_date = comp_daymth + "/" + str(comp_year)

In [ ]:
comp_date

In [ ]:
#Identify earliest "firstfiled" date
firstfiled = ''
for index, row in group_df.iterrows():
    if pd.isnull(row['firstfiled']):
        continue
    else:
        if len(firstfiled) == 0:
            firstfiled = row['firstfiled']
        else:
            continue

In [ ]:
#Identify earliest best date
earliest_BD = ''
for index, row in group_df.iterrows():
    if pd.isnull(row['best_date']):
        continue
    else:
        if len(earliest_BD) == 0:
            earliest_BD = row['best_date']
        else:
            continue

In [ ]:
firstfiled

In [ ]:
earliest_BD

In [ ]:
# Finalize first date variable (minimum of earliest best_date and firstfiled)
if firstfiled =='' and earliest_BD !='':
    first_date = earliest_BD
elif earliest_BD=='' and firstfiled !='':
    first_date = firstfiled
elif firstfiled !='' and earliest_BD !='':
    first_date = min(firstfiled, earliest_BD)

In [ ]:
first_date

In [ ]:
# initiate variables. Groups without these dates are blank for these variables.
BP_date = ''
con_date = ''

#Identify first date for all status categories
m=0
for index, row in group_df.iterrows():
    if m == 0:
        status_previous = 'blah'
    if row['status']=='Building Permit Approved':
        if index == 0:
            BP_date = row['best_date']
        elif index !=0:
            if status_previous =='Building Permit Approved':
                BP_date=BP_date
            else:
                BP_date = row['best_date']
    elif row['status']=='Under Construction':
        if index == 0:
            con_date = row['best_date']
        elif index !=0:
            if status_previous =='Under Construction':
                con_date=con_date
            else:
                con_date = row['best_date']
    status_previous = row['status']
    m=m+1

In [ ]:
print (BP_date, con_date)

In [ ]:
last_row = group_df.tail(1).copy() #most recent row

In [ ]:
last_row

In [ ]:
last_row['latest_project_record_date'] = last_row.best_date
last_row['first_project_record_date'] = group_df.iloc[0].best_date
last_row['latest_project_status'] = last_row.best_stat

In [ ]:
last_row

In [ ]:
## Store a parseable list of all the project states and the dates those states were reported
last_row['project_dates'] = str(tuple(group_df.best_date))
last_row['project_statuses'] = str(tuple(group_df.best_stat))

In [ ]:
last_row

In [ ]:
## Store the project duration in days
latest = last_row.latest_project_record_date.iloc[0]
first = last_row.first_project_record_date.iloc[0]


In [ ]:
if not (pd.isnull(latest) or pd.isnull(first)):
    last_row['project_duration_days'] = (dateutil.parser.parse(latest) - dateutil.parser.parse(first)).days

In [ ]:
last_row

# Actual Code

In [22]:
def convert_to_one_record_per_project(df):
    """
    Group the dataset by (apn, address) and then emit one row per (apn, address) pair.
    best_date and best_stat will be converted into arrays.
    All other attributes will be taken from the record with the most recent `best_date` attribute.
    """
    gb = df.groupby(['apn', 'address'])
    for k in gb.groups:
        group_df = gb.get_group(k)
        group_df = group_df.sort('best_date')
        last_row = group_df.tail(1).copy()
        
        # identify building permit ID. Then fill in rest of quarters with this permit ID.
        building_permit=np.nan
        for index, row in group_df.iterrows():
            if pd.isnull(row['dbi_permit']) and not pd.isnull(building_permit):
                building_permit=building_permit
            else:
                building_permit = row['dbi_permit']
                
        #Identify completion quarter for those projects that have reached completion
        for index, row in group_df.iterrows():
            if row['best_stat'] !='CONSTRUCTION':
                comp_quarter= np.nan
                comp_year=np.nan
            elif row['best_stat'] == 'CONSTRUCTION':
                comp_quarter = row['report_quarter']
                comp_year = row['report_year']

        if pd.isnull(comp_quarter):
            pass
        if comp_quarter == 1 and comp_year == 2016:
            comp_quarter = np.nan
        elif comp_quarter == 4:
            comp_quarter = 1
            comp_year = 1+comp_year
        else:
            comp_quarter = 1+comp_quarter
            
        if comp_quarter ==1:
            comp_daymth= '01/01'
        elif comp_quarter ==2:
            comp_daymth= '04/01'
        elif comp_quarter==3:
            comp_daymth= '07/01'
        elif comp_quarter == 4:
            comp_daymth= '10/01'
        elif pd.isnull(comp_quarter):
            comp_daymth=np.nan

        if pd.isnull(comp_daymth):
            comp_date = np.nan
        else:
            comp_date = comp_daymth + "/" + str(comp_year)
        
        #Identify earliest "firstfiled" date
        firstfiled = ''
        for index, row in group_df.iterrows():
            if pd.isnull(row['firstfiled']):
                continue
            else:
                if len(firstfiled) == 0:
                    firstfiled = row['firstfiled']
                else:
                    continue
        
        #Identify earliest best date
        earliest_BD = ''
        for index, row in group_df.iterrows():
            if pd.isnull(row['best_date']):
                continue
            else:
                if len(earliest_BD) == 0:
                    earliest_BD = row['best_date']
                else:
                    continue
        
        # Finalize first date variable (minimum of earliest best_date and firstfiled)
        if firstfiled =='' and earliest_BD !='':
            first_date = earliest_BD
        elif earliest_BD=='' and firstfiled !='':
            first_date = firstfiled
        elif firstfiled !='' and earliest_BD !='':
            first_date = min(firstfiled, earliest_BD)
            
        # initiate variables. Groups without these dates are blank for these variables.
        BP_date = ''
        con_date = ''

        #Identify first date for all status categories
        m=0
        for index, row in group_df.iterrows():
            if m == 0:
                status_previous = 'blah'
            if row['status']=='Building Permit Approved':
                if index == 0:
                    BP_date = row['best_date']
                elif index !=0:
                    if status_previous =='Building Permit Approved':
                        BP_date=BP_date
                    else:
                        BP_date = row['best_date']
            elif row['status']=='Under Construction':
                if index == 0:
                    con_date = row['best_date']
                elif index !=0:
                    if status_previous =='Under Construction':
                        con_date=con_date
                    else:
                        con_date = row['best_date']
            status_previous = row['status']
            m=m+1
        
        last_row['dbi_permit']= building_permit
        last_row['comp_date']=comp_date
        last_row['BP_date'] = BP_date
        last_row['con_date'] = con_date
        last_row['first_date']=first_date
        last_row['latest_project_record_date'] = last_row.best_date
        last_row['first_project_record_date'] = group_df.iloc[0].best_date
        last_row['latest_project_status'] = last_row.best_stat

        ## Store a parseable list of all the project states and the dates those states were reported
        last_row['project_dates'] = str(tuple(group_df.best_date))
        last_row['project_statuses'] = str(tuple(group_df.best_stat))

        ## Store the project duration in days
        if not (pd.isnull(comp_date) or pd.isnull(first_date)):
            last_row['project_duration_days'] = (dateutil.parser.parse(comp_date) - dateutil.parser.parse(first_date)).days

        yield last_row


def main():
    df = pd.read_csv("cleaned/all_quarters_merged.csv")
    #consolidate status categories. Start with 3 for now
    def status_function(value):
        if value['best_stat']=="CONSTRUCTION":
            field = 'Under Construction'
        elif (value['best_stat']=='BP APPROVED') | (value['best_stat']=='BP ISSUED') | (value['best_stat']=='BP REINSTATED'):
            field = 'Building Permit Approved'
        else:
            field = 'Proposed'
        return field

    df['status']=df.apply(status_function, axis=1)
    
    #keep only variables we want
    list = ['address', 'apn', 'best_date', 'best_stat','status', 'firstfiled', 'report_quarter', 'report_year','units','unitsnet', 'dbi_permit', 'x', 'y']
    df = df[list]
    new_df = pd.concat(convert_to_one_record_per_project(df))
    logging.info("Writing output ({} rows, {} cols) to data/cleaned/all_quarters__one_record_per_project.csv".format(*new_df.shape))
    new_df.to_csv("cleaned/all_quarters__one_record_per_project.csv")
    return new_df

In [23]:
new_df = main()

/Users/briangoggin/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (1,2,6,8,12,13,16,17,20,21,22,23,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,46,48,49,52,53,54,56,61,63,64,65,66,67,68,69,70,72,76,79,80,81,82,83,85,86,87,88,89,90,91,92,93,94,95,100,102,104,105,106,107,108,110,111,112,113,114,117,119,126,129,131,132,133,134,135,136) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/briangoggin/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:10: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [24]:
#First, filter out those projects that are exclusively non-residential (defined as those without units)
new_df = new_df[new_df['units'] > 0]

In [25]:
#Next, keep only those that reached completion at some point over the time period
new_df=new_df[pd.notnull(new_df['comp_date'])]

# Explore Duplicate Data

In [26]:
new_df['address_apn']=new_df['address']+new_df['apn']

In [72]:
#Finally, filter out records with duplicate BP Ids
new_df[new_df.duplicated('dbi_permit', keep=False)]['address_apn'].unique()


array(['1169 MARKET ST3702053', '1634 - 1690 PINE ST0647007',
       '718 LONG BRIDGE ST8710004', '1190 MISSION ST3702052',
       '950 CORBETT AV2799083', '55 LAGUNA ST0857001',
       '1340-1390 MISSION ST3508053', '690 MONTEREY BL3094055',
       '950 CORBETT AV2799013', '443 LINDEN ST0818049',
       '1301 INDIANA ST4228010', '47 MOSS ST3731119',
       'TRINITY PLAZA PHASE III3702391', '5800 03RD ST5431A001',
       'HUNTERS POINT SHIPYARD (UNDER CONSTRUCTION)4591C001',
       '1068 GUERRERO ST3632039', '3150 GEARY BL1066028',
       '49 MOSS ST3731119', '650 INDIANA ST4041009',
       '70 DOUGLASS ST2625007A', '2101 & 2155 WEBSTER STREET0629037',
       'HOPE SF ALICE GRIFFITH4884025', 'MISSION BAY BLOCK 6E8711021',
       'HUDSON AVE AND WHITNEY YOUNG CIR4711118', '70 DOUGLASS ST2625032',
       '790 ANDOVER ST5825031', '2500 ARELIOUS WALKER DR4884025',
       '4 OCTAVIA ST0855011', '5800 03RD ST (BUILDING 4)5431A042',
       '1 ECKER ST3708022', 'TRINITY PLAZA3702391',
       '

In [73]:
new_df[new_df['address_apn']=='1634 - 1690 PINE ST0647007']['dbi_permit']

24161   201312184508.00
Name: dbi_permit, dtype: object

In [76]:
# Same project, different address-apn combination—verified by looking at each address in PIM
new_df[new_df['dbi_permit']==201312184508.00]

,BP_date,address,apn,best_date,best_stat,comp_date,con_date,dbi_permit,first_date,first_project_record_date,firstfiled,latest_project_record_date,latest_project_status,project_dates,project_duration_days,project_statuses,report_quarter,report_year,status,units,unitsnet,x,y,address_apn
24161,,1634 - 1690 PINE ST,0647007,2016-07-01,CONSTRUCTION,07/01/2016,2016-01-14,201312184508.00,2000-11-27,2016-01-14,2000-11-27,2016-07-01,CONSTRUCTION,"('2016-01-14', '2016-03-29', '2016-07-01')",5695.00,"('CONSTRUCTION', 'CONSTRUCTION', 'CONSTRUCTION')",2,2016,Under Construction,260.00,260.00,37.79,-122.42,1634 - 1690 PINE ST0647007
20094,,1634-1690 PINE ST,0647007,2015-10-29,CONSTRUCTION,10/01/2015,2015-07-30,201312184508.00,2000-11-27,2012-07-17,2000-11-27,2015-10-29,CONSTRUCTION,"('2012-07-17', '2012-07-17', '2012-07-17', '20...",5421.00,"('PL FILED', 'PL FILED', 'PL FILED', 'PL FILED...",3,2015,Under Construction,260.00,260.00,37.79,-122.42,1634-1690 PINE ST0647007


In [22]:
#explore duplicated
new_df[new_df['address']=='2155 WEBSTER ST']

,BP_date,address,apn,best_date,best_stat,comp_date,con_date,dbi_permit,first_date,first_project_record_date,firstfiled,latest_project_record_date,latest_project_status,project_dates,project_duration_days,project_statuses,report_quarter,report_year,status,units,unitsnet,x,y
9106,,2155 WEBSTER ST,0629037,2015-03-18,CONSTRUCTION,01/01/2015,2015-03-18,201311222660,2013-11-22,2013-11-22,NaN,2015-03-18,CONSTRUCTION,"('2013-11-22', '2013-11-22', '2013-11-22', '20...",405.0,"('BP FILED', 'BP FILED', 'BP FILED', 'BP FILED...",4,2014,Under Construction,77.0,77.0,37.79057,-122.432337


In [41]:
new_df[new_df['dbi_permit']=='200705010136']

,BP_date,address,apn,best_date,best_stat,comp_date,con_date,dbi_permit,first_date,first_project_record_date,firstfiled,latest_project_record_date,latest_project_status,project_dates,project_duration_days,project_statuses,report_quarter,report_year,status,units,unitsnet,x,y
11114,,72 TOWNSEND ST,3789972,2015-07-21,CONSTRUCTION,07/01/2015,2015-07-21,200705010136,2015-07-21,2015-07-21,NaN,2015-07-21,CONSTRUCTION,"('2015-07-21',)",-20.0,"('CONSTRUCTION',)",2,2015,Under Construction,74.0,74.0,37.781388,-122.390353


In [36]:
new_df[new_df['project_duration_days']<0]

,BP_date,address,apn,best_date,best_stat,comp_date,con_date,dbi_permit,first_date,first_project_record_date,firstfiled,latest_project_record_date,latest_project_status,project_dates,project_duration_days,project_statuses,report_quarter,report_year,status,units,unitsnet,x,y
11114,,72 TOWNSEND ST,3789972,2015-07-21,CONSTRUCTION,07/01/2015,2015-07-21,200705010136,2015-07-21,2015-07-21,NaN,2015-07-21,CONSTRUCTION,"('2015-07-21',)",-20.0,"('CONSTRUCTION',)",2,2015,Under Construction,74.0,74.0,37.781388,-122.390353


# Manual Data Cleaning
Based off of identified duplicates in the previous section

In [54]:
#decided that both this and its duplicate BP at 1169 Market Street are two phases of the same project. Same start date, different end dates
new_df.loc[(new_df['address']=='1190 MISSION ST') & (new_df['apn']=='3702052'), 'first_date'] = '2003-07-15'

In [71]:
#this has a duplicate, same address, different APN
new_df=new_df[(new_df['address']!='3575 GEARY BL') | (new_df['apn']=='1084010')]

In [86]:
#same project, happening at different periods
new_df.loc[(new_df['address']=='1634-1690 PINE ST') & (new_df['apn']=='0647007'), 'comp_date']='07/01/2016'
new_df=new_df[(new_df['address']!='1634 - 1690 PINE ST') | (new_df['apn']=='0647007')]

In [85]:
new_df.loc[(new_df['address']=='1634-1690 PINE ST') & (new_df['apn']=='0647007')]

,BP_date,address,apn,best_date,best_stat,comp_date,con_date,dbi_permit,first_date,first_project_record_date,firstfiled,latest_project_record_date,latest_project_status,project_dates,project_duration_days,project_statuses,report_quarter,report_year,status,units,unitsnet,x,y,address_apn
20094,,1634-1690 PINE ST,0647007,2015-10-29,CONSTRUCTION,07/01/2016,2015-07-30,201312184508.00,2000-11-27,2012-07-17,2000-11-27,2015-10-29,CONSTRUCTION,"('2012-07-17', '2012-07-17', '2012-07-17', '20...",5421.00,"('PL FILED', 'PL FILED', 'PL FILED', 'PL FILED...",3,2015,Under Construction,260.00,260.00,37.79,-122.42,1634-1690 PINE ST0647007


In [ ]:
#Be sure to redo project duration after all manual cleaning